In [1]:
import uproot # type: ignore
import pandas as pd # type: ignore
import numpy as np # type: ignore
import awkward as ak # type: ignore
import matplotlib # type: ignore
from mpl_toolkits.mplot3d import Axes3D # type: ignore
import matplotlib.pyplot as plt # type: ignore
import os
import math
import csv
import seaborn as sns # type: ignore
import sympy as smp # type: ignore
import sympy.stats # type: ignore
from scipy import stats,signal,ndimage,special,fft # type: ignore
from scipy.optimize import curve_fit # type: ignore
from scipy.integrate import quad # type: ignore
from mpl_toolkits.axes_grid1 import make_axes_locatable # type: ignore

Open and read input root file, setup data frame

In [2]:
tracks = uproot.open("/nashome/z/zaemoore/2x2_CAF/2p2h_truth_sample.root")
tracks["TruthTree"].show()
tracks["TruthTree"].keys()

branches = tracks["TruthTree"].arrays()[0]
file_index = branches['file_index']
print("File indeces: ", np.unique(file_index))

name                 | typename                 | interpretation                
---------------------+--------------------------+-------------------------------
true_energy          | std::vector<double>      | AsJagged(AsDtype('>f8'), he...
true_p_x             | std::vector<double>      | AsJagged(AsDtype('>f8'), he...
true_p_y             | std::vector<double>      | AsJagged(AsDtype('>f8'), he...
true_p_z             | std::vector<double>      | AsJagged(AsDtype('>f8'), he...
true_p_mag           | std::vector<double>      | AsJagged(AsDtype('>f8'), he...
true_length          | std::vector<double>      | AsJagged(AsDtype('>f8'), he...
true_angle           | std::vector<double>      | AsJagged(AsDtype('>f8'), he...
true_angle_rot       | std::vector<double>      | AsJagged(AsDtype('>f8'), he...
true_angle_incl      | std::vector<double>      | AsJagged(AsDtype('>f8'), he...
true_angle_x         | std::vector<double>      | AsJagged(AsDtype('>f8'), he...
true_angle_y         | std::

Loop through each file index

In [5]:
p_theta = []

#Function to find angle between 2 (momentum) vectors
#Using dot product
#def theta(p1, p2):
#    dot_product = p1[0] * p2[0] + p1[1] * p2[1] + p1[2] * p2[2]
#    p_theta = np.arccos(dot_product)
#    return p_theta

#Loop through each file

for file_idx in file_index:
    
    true_energy = branches['true_energy'][file_index == file_idx]
    true_p_x = branches['true_p_x'][file_index == file_idx]
    true_p_y = branches['true_p_y'][file_index == file_idx]
    true_p_z = branches['true_p_z'][file_index == file_idx]
    true_p_mag = branches['true_p_mag'][file_index == file_idx]
    true_length = branches['true_length'][file_index == file_idx]
    true_angle = branches['true_angle'][file_index == file_idx]
    true_angle_rot = branches['true_angle_rot'][file_index == file_idx]
    true_angle_incl = branches['true_angle_incl'][file_index == file_idx]
    true_angle_x = branches['true_angle_x'][file_index == file_idx]
    true_angle_y = branches['true_angle_y'][file_index == file_idx]
    true_angle_z = branches['true_angle_z'][file_index == file_idx]
    true_track_start_x = branches['true_track_start_x'][file_index == file_idx]
    true_track_start_y = branches['true_track_start_y'][file_index == file_idx]
    true_track_start_z = branches['true_track_start_z'][file_index == file_idx]
    true_track_end_x = branches['true_track_end_x'][file_index == file_idx]
    true_track_end_y = branches['true_track_end_y'][file_index == file_idx]
    true_track_end_z = branches['true_track_end_z'][file_index == file_idx]
    true_pdg = branches['true_pdg'][file_index == file_idx]
    true_nproton = branches['true_nproton'][file_index == file_idx]
    spill_index = branches['spill_index'][file_index == file_idx]
    interaction_id = branches['interaction_id'][file_index == file_idx]
    event = branches['event'][file_index == file_idx]
    run = branches['run'][file_index == file_idx]
    subrun = branches['subrun'][file_index == file_idx]
    caf_file_name = branches['caf_file_name'][file_index == file_idx]

    #Loop through each event in the file
    for ev in event:
        if all(true_nproton[event == ev]) == 2:
            proton = 2



Plot